# Ingest data

In [28]:
import json
import sys

import druid

task = {
  "type": "index",
  "spec": {
    "dataSchema": {
      "dataSource": "stuff",
      "timestampSpec": {},
      "dimensionsSpec": {},
      "metricsSpec": [
        {
          "type": "doubleSum",
          "name": "value_sum",
          "fieldName": "value"
        }
      ],
      "granularitySpec": {
        "rollup": True
      },
      "transformSpec": {}
    },
    "ioConfig": {
      "type": "index",
      "inputSource": {
        "type": "inline",
        "data": "FILL_IN"
      },
      "inputFormat": {
        "type": "csv",
        "columns": [
          "timestamp",
          "color",
          "value"
        ]
      }
    }
  }
}
task['spec']['ioConfig']['inputSource']['data'] = '\n'.join([
  '2022-01-01T00:01:00.000Z,red,1.0',
  '2022-01-01T00:02:00.000Z,blue,2.0',
  '2022-01-01T00:03:00.000Z,red,3.0',
  '2022-01-01T00:04:00.000Z,blue,4.0'
])
datasource = task["spec"]["dataSchema"]["dataSource"]

task_id = druid.post_task(task)
druid.await_task_completion(task_id, timeout=60)
druid.await_load_completion(datasource, timeout=120)

Task started: index_stuff_gfndhhgo_2022-09-12T21:48:01.468Z
Task log:     http://localhost:8081/druid/indexer/v1/task/index_stuff_gfndhhgo_2022-09-12T21:48:01.468Z/log
Task status:  http://localhost:8081/druid/indexer/v1/task/index_stuff_gfndhhgo_2022-09-12T21:48:01.468Z/status
Task index_stuff_gfndhhgo_2022-09-12T21:48:01.468Z still running...
Task finished with status: SUCCESS
stuff loading complete! You may now query your data


In [29]:
druid.query_sql("""
  select *
  from stuff
""")

[{'__time': '2022-01-01T00:01:00.000Z', 'color': 'red', 'value_sum': 1.0},
 {'__time': '2022-01-01T00:02:00.000Z', 'color': 'blue', 'value_sum': 2.0},
 {'__time': '2022-01-01T00:03:00.000Z', 'color': 'red', 'value_sum': 3.0},
 {'__time': '2022-01-01T00:04:00.000Z', 'color': 'blue', 'value_sum': 4.0}]

# SQL query with `max` and `earliest`

In [30]:
druid.query_sql("""
  select max(value_sum), earliest(value_sum)
  from stuff
""")

[{'EXPR$0': 4.0, 'EXPR$1': 1.0}]

# Define native queries

In [31]:
inner_query = {
  "dataSource": "stuff",
  "queryType": "timeseries",
  "intervals": [
    "2022-01-01T00:00:00.000Z/PT5M"
  ],
  "granularity": "all",
  "aggregations": [
    {
      "type": "doubleFirst",
      "name": "earliest",
      "fieldName": "value_sum"
    },
    {
      "type": "doubleMax",
      "name": "max",
      "fieldName": "value_sum"
    }
  ]
}

nested_query = {
  "dataSource": {
    "type": "query",
    "query": inner_query
  },
  "queryType": "timeseries",
  "granularity": "all",
  "aggregations": [
    {
      "type": "doubleAny",
      "name": "earliest",
      "fieldName": "earliest"
    },
    {
      "type": "doubleAny",
      "name": "max",
      "fieldName": "max"
    },
  ],
  "intervals": [ "2022-01-01T00:00:00.000Z/PT5M" ],
  "dimensions": []
}

# Native query: `timeseries`

In [32]:
print(f'Inner:  {druid.query(inner_query)}')
print(f'Nested: {druid.query(nested_query)}')

Inner:  [{'timestamp': '2022-01-01T00:01:00.000Z', 'result': {'earliest': 1.0, 'max': 4.0}}]
Nested: [{'timestamp': '2022-01-01T00:00:00.000Z', 'result': {'max': 4.0, 'earliest': 1.0}}]


# Native query: `groupBy`

In [33]:
inner_query['queryType'] = 'groupBy'
nested_query['queryType']= 'groupBy'
nested_query['dataSource']['query'] = inner_query

print(f'Inner:  {druid.query(inner_query)}')
print(f'Nested: {druid.query(nested_query)}')

Inner:  [{'version': 'v1', 'timestamp': '2022-01-01T00:00:00.000Z', 'event': {'max': 4.0, 'earliest': 1.0}}]
Nested: [{'version': 'v1', 'timestamp': '2022-01-01T00:00:00.000Z', 'event': {'max': 4.0, 'earliest': 0.0}}]


### <span style="color:red">Note the `earliest: 0.0` in the result</span>👆

# Native query: `groupBy` with `stringAny` outer agg

In [34]:
for agg in nested_query['aggregations']:
  agg['type'] = 'stringAny'

print(f'Inner:  {druid.query(inner_query)}')
print(f'Nested: {druid.query(nested_query)}')


Inner:  [{'version': 'v1', 'timestamp': '2022-01-01T00:00:00.000Z', 'event': {'max': 4.0, 'earliest': 1.0}}]
Nested: [{'version': 'v1', 'timestamp': '2022-01-01T00:00:00.000Z', 'event': {'max': '4.0', 'earliest': 'Pair{lhs=1640995260000, rhs=1.0}'}}]


### Note the `Pair` in the result 👆
